In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor
import pickle
import os
import matplotlib.pyplot as plt
import difflib

# Load and process the Mendalay.csv dataset
def load_mendalay_data(file_path="Mendalay.csv"):
    try:
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Check if 'Timestamp' column exists
        if 'Timestamp' not in df.columns:
            # Rename the first column to 'Timestamp'
            df = pd.read_csv(file_path, header=0)
            df.columns.values[0] = 'Timestamp'
        
        # Convert Timestamp to datetime format
        try:
            # Try DD/MM/YYYY format first
            df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%d/%m/%Y %H:%M:%S')
        except:
            try:
                # Try MM/DD/YYYY format if the first fails
                df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%m/%d/%Y %H:%M:%S')
            except:
                # Use pandas' flexible parser as a fallback
                df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        
        # Set Timestamp as index
        df.set_index('Timestamp', inplace=True)
        
        # Ensure column names match what we need
        column_mapping = {
            'Temperature [Â°C]': 'Temperature [°C]',
            'Temperature [°C]': 'Temperature [°C]',
            'Relative_Humidity [%]': 'Relative_Humidity [%]',
            'Relative Humidity [%]': 'Relative_Humidity [%]',
            'RelativeHumidity [%]': 'Relative_Humidity [%]'
        }
        
        df.rename(columns=column_mapping, inplace=True)
        
        # Handle missing values if any
        df.fillna(method='ffill', inplace=True)
        
        print(f"Loaded data shape: {df.shape}")
        print(f"Columns: {df.columns.tolist()}")
        print(f"Date range: {df.index.min()} to {df.index.max()}")
        
        return df
    
    except Exception as e:
        print(f"Error loading data: {e}")
        raise

# Create features and targets for time series forecasting
def create_features_and_targets(df, n_past=5, n_future=5):
    X, y = [], []
    
    # Extract the columns we want to use
    temp_data = df['Temperature [°C]'].values
    humid_data = df['Relative_Humidity [%]'].values
    
    # Loop through the data to create sequences
    for i in range(len(df) - n_past - n_future + 1):
        # Past data (features)
        past_temp = temp_data[i:i+n_past]
        past_humid = humid_data[i:i+n_past]
        
        # Combine into a single feature vector
        past_features = np.concatenate([past_temp, past_humid])
        X.append(past_features)
        
        # Future data (targets)
        future_temp = temp_data[i+n_past:i+n_past+n_future]
        future_humid = humid_data[i+n_past:i+n_past+n_future]
        
        # Combine into a single target vector
        future_targets = np.concatenate([future_temp, future_humid])
        y.append(future_targets)
    
    return np.array(X), np.array(y)

# Train the forecasting model
def build_forecasting_model(df, n_past=5, n_future=5):
    # Create features and targets
    X, y = create_features_and_targets(df, n_past, n_future)
    
    print(f"Feature shape: {X.shape}, Target shape: {y.shape}")
    
    # Normalize the data
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y)
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_scaled, test_size=0.2, random_state=42
    )
    
    # Create and train the model
    base_model = XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1)
    model = MultiOutputRegressor(base_model)
    
    print("Training the model...")
    model.fit(X_train, y_train)
    
    # Evaluate the model
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    
    print(f"Mean Squared Error: {mse:.4f}")
    print(f"Mean Absolute Error: {mae:.4f}")
    
    # Calculate metrics for temperature and humidity separately
    temp_mse = mean_squared_error(y_test[:, :n_future], y_pred[:, :n_future])
    humid_mse = mean_squared_error(y_test[:, n_future:], y_pred[:, n_future:])
    
    print(f"Temperature MSE: {temp_mse:.4f}")
    print(f"Humidity MSE: {humid_mse:.4f}")
    
    return model, scaler_X, scaler_y

# Save the model and scalers
def save_model(model, scaler_X, scaler_y, n_past, n_future, prefix="mendalay_forecast"):
    # Create directory if it doesn't exist
    if not os.path.exists('models'):
        os.makedirs('models')
    
    # Save model
    with open(f'models/{prefix}_model.pkl', 'wb') as f:
        pickle.dump(model, f)
    
    # Save scalers
    with open(f'models/{prefix}_scaler_X.pkl', 'wb') as f:
        pickle.dump(scaler_X, f)
    
    with open(f'models/{prefix}_scaler_y.pkl', 'wb') as f:
        pickle.dump(scaler_y, f)
        
    # Save the configuration (n_past, n_future)
    config = {
        'n_past': n_past,
        'n_future': n_future
    }
    
    with open(f'models/{prefix}_config.pkl', 'wb') as f:
        pickle.dump(config, f)
    
    print(f"Model, scalers, and configuration saved to 'models/' directory with prefix '{prefix}'")

# Train and save models for both 5-hour and 24-hour forecasting
def main():
    # Load data
    print("Loading data from Mendalay.csv...")
    data = load_mendalay_data()
    
    # Keep only temperature and humidity columns
    required_columns = ['Temperature [°C]', 'Relative_Humidity [%]']
    
    # Check if these columns exist in the data
    for col in required_columns:
        if col not in data.columns:
            closest_match = difflib.get_close_matches(col, data.columns, n=1)
            if closest_match:
                print(f"Column '{col}' not found, using '{closest_match[0]}' instead")
                data.rename(columns={closest_match[0]: col}, inplace=True)
            else:
                print(f"Column '{col}' not found and no close match available")
                return
    
    data = data[required_columns]
    
    # Build and train 5-hour model
    print("\n=== Training 5-hour forecast model ===")
    model_5h, scaler_X_5h, scaler_y_5h = build_forecasting_model(data, n_past=5, n_future=5)
    
    # Save 5-hour model
    save_model(model_5h, scaler_X_5h, scaler_y_5h, n_past=5, n_future=5, prefix="mendalay_forecast_5h")
    
    # Build and train 24-hour model
    print("\n=== Training 24-hour forecast model ===")
    model_24h, scaler_X_24h, scaler_y_24h = build_forecasting_model(data, n_past=24, n_future=24)
    
    # Save 24-hour model
    save_model(model_24h, scaler_X_24h, scaler_y_24h, n_past=24, n_future=24, prefix="mendalay_forecast_24h")
    
    print("\nTraining complete. Models saved successfully!")

if __name__ == "__main__":
    main()

Loading data from Mendalay.csv...
Loaded data shape: (11263, 6)
Columns: ['PM2.5 [ug/m3]', 'PM10[ug/m3]', 'Ozone [ppb]', 'Carbon_Monoxide [ppb]', 'Temperature [°C]', 'Relative_Humidity [%]']
Date range: 2022-01-31 12:00:00 to 2023-05-30 23:00:00

=== Training 5-hour forecast model ===
Feature shape: (11254, 10), Target shape: (11254, 10)
Training the model...


C:\Users\User\AppData\Local\Temp\ipykernel_2364\1707769888.py:52: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


Mean Squared Error: 0.1793
Mean Absolute Error: 0.2928
Temperature MSE: 0.1886
Humidity MSE: 0.1701
Model, scalers, and configuration saved to 'models/' directory with prefix 'mendalay_forecast_5h'

=== Training 24-hour forecast model ===
Feature shape: (11216, 48), Target shape: (11216, 48)
Training the model...
Mean Squared Error: 0.1819
Mean Absolute Error: 0.2960
Temperature MSE: 0.1338
Humidity MSE: 0.2300
Model, scalers, and configuration saved to 'models/' directory with prefix 'mendalay_forecast_24h'

Training complete. Models saved successfully!


LOADING THE MODEL

In [ ]:
import pandas as pd
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Load the model and scalers
def load_model(mode="5h"):
    """
    Load the trained model, scalers, and configuration
    
    Parameters:
    mode: Either "5h" or "24h" to load the appropriate model
    
    Returns:
    model: Trained forecasting model
    scaler_X: Feature scaler
    scaler_y: Target scaler
    config: Model configuration
    """
    prefix = f"mendalay_forecast_{mode}"
    
    try:
        # Load model
        with open(f'models/{prefix}_model.pkl', 'rb') as f:
            model = pickle.load(f)
        
        # Load scalers
        with open(f'models/{prefix}_scaler_X.pkl', 'rb') as f:
            scaler_X = pickle.load(f)
        
        with open(f'models/{prefix}_scaler_y.pkl', 'rb') as f:
            scaler_y = pickle.load(f)
            
        # Load configuration
        with open(f'models/{prefix}_config.pkl', 'rb') as f:
            config = pickle.load(f)
        
        print(f"Model and scalers loaded successfully for {mode} forecasting")
        
        return model, scaler_X, scaler_y, config
    
    except Exception as e:
        print(f"Error loading model: {e}")
        print("Please make sure you've run the training script first.")
        return None, None, None, None

# Function to get user input
def get_user_input(n_past):
    """
    Get temperature and humidity values from the user
    
    Parameters:
    n_past: Number of past hours to collect data for
    
    Returns:
    temperatures: List of temperature values
    humidities: List of humidity values
    start_time: Starting timestamp for forecasting
    """
    print(f"\nPlease enter temperature and humidity data for the past {n_past} hours.")
    print("Start with the oldest data point and move forward in time.")
    
    temperatures = []
    humidities = []
    
    for i in range(n_past):
        hour_label = f"{i+1}/{n_past}"
        
        while True:
            try:
                temp = float(input(f"Hour {hour_label} - Temperature [°C]: "))
                humid = float(input(f"Hour {hour_label} - Relative Humidity [%]: "))
                break
            except ValueError:
                print("Please enter valid numerical values.")
        
        temperatures.append(temp)
        humidities.append(humid)
    
    # Get the timestamp for the most recent data point
    while True:
        try:
            time_str = input("\nEnter the timestamp for the most recent data point (YYYY-MM-DD HH:MM): ")
            start_time = datetime.strptime(time_str, "%Y-%m-%d %H:%M")
            break
        except ValueError:
            print("Invalid date format. Please use YYYY-MM-DD HH:MM format.")
    
    return temperatures, humidities, start_time

# Function to make predictions
def forecast_weather(model, scaler_X, scaler_y, temps, humids, n_past, n_future):
    """
    Forecasts the next n_future hours based on input data
    
    Parameters:
    model: Trained forecasting model
    scaler_X: Feature scaler
    scaler_y: Target scaler
    temps: List of temperature values
    humids: List of humidity values
    n_past: Number of past time steps used as input features
    n_future: Number of future time steps to predict
    
    Returns:
    temp_forecast: Forecasted temperature values
    humid_forecast: Forecasted humidity values
    """
    # Combine into feature vector
    features = np.concatenate([temps, humids]).reshape(1, -1)
    
    # Scale features
    features_scaled = scaler_X.transform(features)
    
    # Make prediction
    forecast_scaled = model.predict(features_scaled)
    
    # Inverse transform
    forecast = scaler_y.inverse_transform(forecast_scaled)
    
    # Separate temperature and humidity forecasts
    temp_forecast = forecast[0][:n_future]
    humid_forecast = forecast[0][n_future:]
    
    return temp_forecast, humid_forecast

# Function to display results
def display_results(temps, humids, temp_forecast, humid_forecast, start_time, n_past, n_future):
    """
    Display the input data and forecast results
    
    Parameters:
    temps: Input temperature values
    humids: Input humidity values
    temp_forecast: Forecasted temperature values
    humid_forecast: Forecasted humidity values
    start_time: Starting timestamp for the most recent input data
    n_past: Number of past time steps used as input features
    n_future: Number of future time steps predicted
    """
    # Create timestamps for past data
    past_times = [(start_time - timedelta(hours=n_past-i-1)) for i in range(n_past)]
    
    # Create timestamps for future data
    future_times = [(start_time + timedelta(hours=i+1)) for i in range(n_future)]
    
    # Print tabular results
    print("\n===== INPUT DATA =====")
    print("Timestamp               | Temperature (°C) | Humidity (%)")
    print("-" * 60)
    for i in range(n_past):
        time_str = past_times[i].strftime("%Y-%m-%d %H:%M")
        print(f"{time_str} | {temps[i]:15.2f} | {humids[i]:11.2f}")
    
    print("\n===== FORECAST RESULTS =====")
    print("Timestamp               | Temperature (°C) | Humidity (%)")
    print("-" * 60)
    for i in range(n_future):
        time_str = future_times[i].strftime("%Y-%m-%d %H:%M")
        print(f"{time_str} | {temp_forecast[i]:15.2f} | {humid_forecast[i]:11.2f}")
    
    # Create DataFrames for plotting
    past_df = pd.DataFrame({
        'Timestamp': past_times,
        'Temperature [°C]': temps,
        'Relative_Humidity [%]': humids
    })
    past_df.set_index('Timestamp', inplace=True)
    
    future_df = pd.DataFrame({
        'Timestamp': future_times,
        'Temperature [°C]': temp_forecast,
        'Relative_Humidity [%]': humid_forecast
    })
    future_df.set_index('Timestamp', inplace=True)
    
    # Plot the results
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    
    # Plot temperature
    ax1.plot(past_df.index, past_df['Temperature [°C]'], 'b-o', label='Input Temperature')
    ax1.plot(future_df.index, future_df['Temperature [°C]'], 'r--o', label='Forecasted Temperature')
    ax1.set_title('Temperature Forecast')
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Temperature [°C]')
    ax1.legend()
    ax1.grid(True)
    
    # Plot humidity
    ax2.plot(past_df.index, past_df['Relative_Humidity [%]'], 'g-o', label='Input Humidity')
    ax2.plot(future_df.index, future_df['Relative_Humidity [%]'], 'm--o', label='Forecasted Humidity')
    ax2.set_title('Relative Humidity Forecast')
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Relative Humidity [%]')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Save the forecast to a CSV file
    combined_df = pd.concat([past_df, future_df])
    combined_df.to_csv('forecast_results.csv')
    print("\nForecast saved to 'forecast_results.csv'")

# Main function
def main():
    print("======================================")
    print("Weather Forecasting Prediction System")
    print("======================================")
    
    # Ask user which forecasting mode they want
    while True:
        mode = input("\nChoose forecasting mode (5h or 24h): ").strip().lower()
        if mode in ['5h', '24h']:
            break
        print("Invalid choice. Please enter '5h' or '24h'.")
    
    # Load the appropriate model
    model, scaler_X, scaler_y, config = load_model(mode)
    
    if model is None:
        return
    
    n_past = config['n_past']
    n_future = config['n_future']
    
    print(f"\nLoaded {mode} forecasting model (using past {n_past} hours to predict next {n_future} hours)")
    
    # Get user input
    temps, humids, start_time = get_user_input(n_past)
    
    # Make prediction
    temp_forecast, humid_forecast = forecast_weather(
        model, scaler_X, scaler_y, temps, humids, n_past, n_future
    )
    
    # Display results
    display_results(temps, humids, temp_forecast, humid_forecast, start_time, n_past, n_future)

if __name__ == "__main__":
    main()